In [12]:
from http.server import HTTPServer, BaseHTTPRequestHandler
from threading import Thread
from time import sleep
import requests
import ipywidgets as widgets
from IPython.display import HTML, display

In [13]:
out = widgets.Output(layout={'border': '1px solid black'})

In [14]:
out.clear_output()

In [15]:
out

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [16]:
with out:
    display(HTML("<h1>Request Headers</h1>"))

In [17]:
with out:
    print("hello")

In [18]:
class PrintHandler(BaseHTTPRequestHandler):
    
    counter = 1
    
    def do_GET(self):
        with out:
            print(self.headers)
        self.send_response(200)
        self.end_headers()
        self.wfile.write(bytes(f"Hello, world {self.counter}", "utf-8"))
        PrintHandler.counter += 1
httpd = HTTPServer(('localhost', 8000), PrintHandler)

OSError: [Errno 48] Address already in use

In [7]:
t = Thread(target=httpd.serve_forever)

In [8]:
t.start()

In [9]:
r = requests.get("http://localhost:8000")
r.text

127.0.0.1 - - [11/Aug/2023 13:13:44] "GET / HTTP/1.1" 200 -


'Hello, world 1'

In [10]:
t.join()

KeyboardInterrupt: 

In [ ]:
error = None
while True:
    try:
        r = requests.get('https://www.google.com')
        r.raise_for_status()
        sleep(1)
    except Exception as e:
        error = e
        break

In [ ]:
error2 = error.args[0]

In [ ]:
error2.args